In [1]:
import sys
sys.path.append("../scripts")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from clock_utils import make_groups

%load_ext autoreload
%autoreload 2

basepath = "../data/paper_data/"

In [2]:
# Get preprocessed data (already log scaled)
data_crct_none = pd.read_csv(basepath + "data_atac_crct_none.tsv", index_col=0, sep="\t")
data_crct_sex = pd.read_csv(basepath + "data_atac_crct_sex.tsv", index_col=0, sep="\t")
# Get meta
meta = pd.read_csv(basepath + "meta_final.tsv", sep="\t")

# Subset all tables to common samples
csamps = data_crct_sex.index.values
data_crct_none = data_crct_none.loc[csamps, ]
data_crct_sex = data_crct_sex.loc[csamps, ]

# Get ages for common samples
meta = meta.set_index("Subject", drop=False)
labels = meta.loc[csamps, "Age"]

groups = make_groups(11, labels)

In [4]:
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipes = list()

# Model 1

pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("regressor", ElasticNet(max_iter=5000, tol=0.0005))])

param_grid = dict()
param_grid["regressor__alpha"] = np.logspace(-4, 1, 30) #40
param_grid["regressor__l1_ratio"] = [0.7, 0.8, 0.9, 0.95, 0.99, 1]
# param_grid["regressor__alpha"] = np.logspace(-4, 1, 10) #40
# param_grid["regressor__l1_ratio"] = [0.95]

In [5]:
from sklearn.model_selection import LeavePGroupsOut

outer_groups_out = 1
scoring = "neg_median_absolute_error"
cv_outer = LeavePGroupsOut(outer_groups_out)
outer_split = list(cv_outer.split(data_crct_none, labels, groups)) # Can be applied to both datasets

In [6]:
import shelve, os, dbm.dumb
from datetime import datetime

date = datetime.now().strftime("%Y-%m-%d_%H-%M")
outpath = "../clocks/parallel/" + date + "_crct_none_vs_sex"
os.makedirs(outpath, exist_ok=True)

data = data_crct_none
vars_to_save = ["data", "labels", "groups", "pipe", "param_grid", "outer_split", "scoring"]

dumbdb = dbm.dumb.open(outpath + "/dataset")
dataset = shelve.Shelf(dumbdb)
for key in vars_to_save:
    try:
        dataset[key] = globals()[key]
        print('Shelved {0}'.format(key))
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
dataset.close()
dumbdb.close()

Shelved data
Shelved labels
Shelved groups
Shelved pipe
Shelved param_grid
Shelved outer_split
Shelved scoring


In [7]:
date = datetime.now().strftime("%Y-%m-%d_%H-%M")
outpath = "../clocks/parallel/" + date + "_crct_sex_vs_none"
os.makedirs(outpath, exist_ok=True)

data = data_crct_sex
vars_to_save = ["data", "labels", "groups", "pipe", "param_grid", "outer_split", "scoring"]

dumbdb = dbm.dumb.open(outpath + "/dataset")
dataset = shelve.Shelf(dumbdb)
for key in vars_to_save:
    try:
        dataset[key] = globals()[key]
        print('Shelved {0}'.format(key))
    except TypeError:
        #
        # __builtins__, my_shelf, and imported modules can not be shelved.
        #
        print('ERROR shelving: {0}'.format(key))
dataset.close()
dumbdb.close()

Shelved data
Shelved labels
Shelved groups
Shelved pipe
Shelved param_grid
Shelved outer_split
Shelved scoring
